# OS11 - Exercises 0-6

In [1]:
from sympy import *

In [2]:
import numpy as np

In [3]:
import scipy.linalg as linalg

In [4]:
from scipy.optimize import linprog

In [5]:
from itertools import *

## Exercise 0

Build a function that transforms a Node-Node matrix to a Node-Arc matrix.

### Data Introduction

Data will be introduced on a list of (origin, dest, cost) tuples (or any indexable).

In [6]:
datos_originales = []

In [7]:
datos_originales.append((1, 2, 1))
#datos_originales.append((2, 1, 1))   #creo un ciclo a proposito #no es una buena idea
datos_originales.append((1, 3, 1))
datos_originales.append((2, 3, 1))
datos_originales.append((3, 4, 1))
datos_originales.append((4, 1, 1))


In [8]:
datos_originales

[(1, 2, 1), (1, 3, 1), (2, 3, 1), (3, 4, 1), (4, 1, 1)]

### Node-Node Matrix creation

We need to pass from a sparse to a dense matrix formulation.

In [9]:
#Max nodes:
nodes = 0
for d in datos_originales:
    if d[0]> nodes:
        nodes =d[0]
    if d[1] > nodes:
        nodes = d[1]

In [10]:
nodes

4

In [11]:
NN = np.zeros((4,4))

In [12]:
NN

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [13]:
for d in datos_originales:
    NN[d[0]-1, d[1]-1] = d[2]

In [14]:
NN

array([[0., 1., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

### Arc Detection

We create a funtion that returns a list of arcs and weights from NN matrix. Basically reverts to original data.

In [15]:
def arc_detection(NN):
    arcs = []
    #each arc will be represented as a tuple
    for a in ((x,y) for x in range(NN.shape[0]) for y in range(NN.shape[1])):
        if NN[a]>0:
            arcs.append((a, NN[a]))
    return arcs

In [16]:
arcs = arc_detection(NN)

In [17]:
arcs

[((0, 1), 1.0), ((0, 2), 1.0), ((1, 2), 1.0), ((2, 3), 1.0), ((3, 0), 1.0)]

### Arc-Nodes Matrix Creation

Finally we created ANMatrix funtion that receives a n N x N matrix and returns a Nodes x Arcs Matrix.

In [18]:
def ANMatrix(NN):
    arcs = arc_detection(NN)
    MN = np.zeros((NN.shape[0], len(arcs))).astype(int)
    for a in range(len(arcs)):
        MN[arcs[a][0][0], a] = arcs[a][1]
        MN[arcs[a][0][1], a] = -arcs[a][1]
    arcss = [a[0] for a in arcs]
    return MN, arcss

In [19]:
ANMatrix(NN)

(array([[ 1,  1,  0,  0, -1],
        [-1,  0,  1,  0,  0],
        [ 0, -1, -1,  1,  0],
        [ 0,  0,  0, -1,  1]]),
 [(0, 1), (0, 2), (1, 2), (2, 3), (3, 0)])

## Ex. 01-02

Data introduction: we load the graph using Ex. 0 technique.

In [20]:
named_nodes = ['s', 2, 3, 4, 5, 't']   #node enumeration. Can be any non repeated inmutable.

I want to use index notation on node names, for this we can create a dictionary with node names and indexes.

In [21]:
def nodes_indexer(named_nodes):
    nodes = {}
    for i in range(len(named_nodes)):
        nodes[named_nodes[i]] = i
    return nodes

In [22]:
nodes = nodes_indexer(named_nodes)

Example:

In [23]:
nodes['t']

5

In [24]:
nodes[5]

4

Using nodes indexed, I load oriented graph data using Ex. 0 schema.

In [25]:
graph = []

graph.append(((nodes['s'], nodes[2]), 2))
graph.append(((nodes['s'], nodes[3]), 2))
graph.append(((nodes[2], nodes[4]), 2))
graph.append(((nodes[2], nodes['t']), 5))
graph.append(((nodes[3], nodes[5]), 2))
graph.append(((nodes[4], nodes['t']), 1))
graph.append(((nodes[5], nodes['t']), 2))

graph


[((0, 1), 2),
 ((0, 2), 2),
 ((1, 3), 2),
 ((1, 5), 5),
 ((2, 4), 2),
 ((3, 5), 1),
 ((4, 5), 2)]

I'll construct a new Arc-Nodes matrix function that uses directly graph data.

In [26]:
def ANMatrix_from_arcs(graph, num_nodes):
    num_arcs = len(graph)
    arcs = []
    MN = np.zeros((num_nodes, num_arcs)).astype(int)
    for a in range(len(graph)):
        MN[graph[a][0][0], a] = 1
        MN[graph[a][0][1], a] = -1
        arcs.append(graph[a][0])
    return MN, arcs

In [27]:
res = ANMatrix_from_arcs(graph, len(nodes))
res

(array([[ 1,  1,  0,  0,  0,  0,  0],
        [-1,  0,  1,  1,  0,  0,  0],
        [ 0, -1,  0,  0,  1,  0,  0],
        [ 0,  0, -1,  0,  0,  1,  0],
        [ 0,  0,  0,  0, -1,  0,  1],
        [ 0,  0,  0, -1,  0, -1, -1]]),
 [(0, 1), (0, 2), (1, 3), (1, 5), (2, 4), (3, 5), (4, 5)])

In [28]:
MAN = res[0]
arcos = res[1]

Necesitamos construir el vector de costos (longitudes entre nodos).

In [29]:
def costos(graph):
    c = []
    for a in graph:
        c.append(a[1])
    return c

In [30]:
c = costos(graph)
c = np.array(c)
c

array([2, 2, 2, 5, 2, 1, 2])

El verctor de igualdades b:
    

In [31]:
b = np.zeros((len(nodes)))
b

array([0., 0., 0., 0., 0., 0.])

Asignamos los valores predeterminados de los nodos de entrada y salida:

In [32]:
b[nodes['s']] = 1
b[nodes['t']] = -1

In [33]:
b

array([ 1.,  0.,  0.,  0.,  0., -1.])

In [34]:
limites = [(0, None) for a in arcos]

In [35]:
limites

[(0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None)]

In [36]:
r = linprog(c=c, A_eq = MAN, b_eq = b, bounds = limites, method = 'revised simplex')

<ipython-input-36-a72eea7bce7e>:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  r = linprog(c=c, A_eq = MAN, b_eq = b, bounds = limites, method = 'revised simplex')


In [37]:
resultados = r['x']

In [38]:
resultados

array([1., 0., 1., 0., 0., 1., 0.])

El costo total:

In [39]:
r['fun']

5.0

Los arcos seleccionados:

In [40]:
camino = []
for a in range(len(graph)):
    if resultados[a] > 0:
        camino.append(graph[a])

In [41]:
camino

[((0, 1), 2), ((1, 3), 2), ((3, 5), 1)]

In [42]:
def print_camino(camino):
    s = ''
    sep = ' -> '
    if len(camino)>0:
        s += str(camino[0][0][0]) + sep
    for t in camino:
        s += str(t[0][1]) + sep
    return s

In [43]:
print_camino(camino)

'0 -> 1 -> 3 -> 5 -> '

## Ex 03 - Dikjstra

### Data estructure definition

For this exercice, I will use a class as Node type. Each node contain neighbours information and also, after Dijkstra algorithm, optimum  previous node and total distance from origin.

In [44]:
class Nodo:
    def __init__(self, nombre):
        self.nombre = nombre     #identifier. Any non mutable
        self.distancia = None    #distance from origin
        self.previo = None   #previos node
        self.vecinos = {}    #dictionary of neighbour, distance pairs (forward)
    
    def agregar_vecino(self, nodo, distancia):
        self.vecinos[nodo] = distancia
        
    def camino(self):
        #Constructs an string containing path from node to origin.
        s = str(self.nombre) + ' <- ' + (self.previo.camino() if self.previo else '')
        return s
    
    #usual python duck-typing:
    def __str__(self):
        return "Nodo {0} ".format(self.nombre, self.distancia, self.previo)
    
    def __repr__(self):
        return self.__str__()
    


### Graph data

Using preceden struct, I construc the graph

In [45]:
Nodos = {}

Nodos['s'] = Nodo('s')
Nodos[2] = Nodo(2)
Nodos[3] = Nodo(3)
Nodos[4] = Nodo(4)
Nodos[5] = Nodo(5)
Nodos['t'] = Nodo('t')

In [46]:
Nodos['s'].agregar_vecino(Nodos[2], 2)
Nodos['s'].agregar_vecino(Nodos[3], 2)

Nodos[2].agregar_vecino(Nodos[4], 2)
Nodos[2].agregar_vecino(Nodos['t'], 5)

Nodos[3].agregar_vecino(Nodos[5], 2)

Nodos[4].agregar_vecino(Nodos['t'], 1)

Nodos[5].agregar_vecino(Nodos['t'], 2)


Example usage...

In [47]:
Nodos['s']

Nodo s 

In [48]:
Nodos[2].vecinos

{Nodo 4 : 2, Nodo t : 5}

### Dijkstra algoritm

I implement in a single function the algorithm.

In [49]:
def algoritmo_Dijkstra(nodo_inicial):
    modificados = []
    nodo_inicial.distancia = 0
    seguir = True
    modificados.append(nodo_inicial)    # on first iteration, initial node is marked as modified
    while seguir:
        new_modif = []   #each iteration, we record modified nodes
        for nodo in modificados:   #for each node modified on previous iteration
            for v in nodo.vecinos: #for each node in neigborhoods
                if not v.distancia:     #if it's first time in this node
                    v.distancia = nodo.distancia + nodo.vecinos[v]  # new distance
                    v.previo = nodo
                    if not v in new_modif:
                        new_modif.append(v) 
                else:
                    d_p = nodo.distancia + nodo.vecinos[v]  # new distance candidate
                    if d_p < v.distancia:
                        v.previo = nodo
                        v.distancia = d_p
                        if not v in new_modif:
                            new_modif.append(v) 
        if len(new_modif) > 0:
            modificados = new_modif
            seguir = True
        else:
            seguir = False

### Using

In [50]:
algoritmo_Dijkstra(Nodos['s'])

In [51]:
Nodos['t'].distancia

5

In [52]:
Nodos['t'].camino()

't <- 4 <- 2 <- s <- '

## Ex 4 - Maximum Flow

In [53]:
named_nodes = ['s', 2, 3, 4, 5, 't']   #node enumeration. Can be any non repeated inmutable.

In [54]:
nodes = nodes_indexer(named_nodes)

In [55]:
graph = []

graph.append(((nodes['s'], nodes[2]), 7))
graph.append(((nodes['s'], nodes[3]), 1))

graph.append(((nodes[2], nodes[4]), 2))
graph.append(((nodes[2], nodes['t']), 3))

graph.append(((nodes[3], nodes[5]), 2))

graph.append(((nodes[4], nodes['t']), 1))

graph.append(((nodes[5], nodes['t']), 2))

graph.append(((nodes['t'], nodes['s']), None))


graph

[((0, 1), 7),
 ((0, 2), 1),
 ((1, 3), 2),
 ((1, 5), 3),
 ((2, 4), 2),
 ((3, 5), 1),
 ((4, 5), 2),
 ((5, 0), None)]

In [56]:
res = ANMatrix_from_arcs(graph, len(nodes))
A = res[0]
arcos = res[1]

In [57]:
A

array([[ 1,  1,  0,  0,  0,  0,  0, -1],
       [-1,  0,  1,  1,  0,  0,  0,  0],
       [ 0, -1,  0,  0,  1,  0,  0,  0],
       [ 0,  0, -1,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  1,  0],
       [ 0,  0,  0, -1,  0, -1, -1,  1]])

In [58]:
arcos

[(0, 1), (0, 2), (1, 3), (1, 5), (2, 4), (3, 5), (4, 5), (5, 0)]

In [59]:
c = np.zeros((len(arcos)))
c[-1]=-1
c

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.])

El verctor de igualdades b:
    

In [60]:
b = np.zeros((len(nodes)))
b

array([0., 0., 0., 0., 0., 0.])

In [61]:
limites = [(0, a[1]) for a in graph]

In [62]:
limites

[(0, 7), (0, 1), (0, 2), (0, 3), (0, 2), (0, 1), (0, 2), (0, None)]

In [63]:
r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, method='revised simplex')

<ipython-input-63-62916b0d23f2>:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, method='revised simplex')


In [64]:
resultados = r['x']

In [65]:
resultados

array([4., 1., 1., 3., 1., 1., 1., 5.])

In [66]:
resultados[-1]

5.0

## Ex 5 - Ford-Fulkenson Algorithm

### Data estructure definition

In [67]:
class Nodo:
    def __init__(self, nombre):
        self.nombre = nombre     #identifier. Any non mutable
        self.branches = {}    #dictionary of neighbour, distance pairs (forward)
    
    def add_branch(self, nodo, max_flow):
        self.branches[nodo] = {'max_flow': max_flow, 'available_flow': max_flow, 'current_flow':0.0}
        
    #usual python duck-typing:
    def __str__(self):
        return "Node {0} ".format(self.nombre)
    
    def report(self):
        s = 'Node {0} - Branches: {1}\n'.format(self.nombre, len(self.branches))
        total = 0.0
        available = 0.0
        current = 0.0
        for k, v in self.branches.items():
            total += v['max_flow']
            available += v['available_flow']
            current += v['current_flow']
            s += "    Branch {0} - Max Flow {1} - Current Flow {2} - Available Flow {3}\n".format(str(k), v['max_flow'], v['current_flow'], v['available_flow'])
        
        s += "Node Stats: Max Flow {0} - Current Flow {1} - Available Flow {2}\n".format(total, current, available)
        return s
    
    def __repr__(self):
        return self.__str__()
    


### Graph Data

In [68]:
Nodos = {}

Nodos['s'] = Nodo('s')
Nodos[2] = Nodo(2)
Nodos[3] = Nodo(3)
Nodos[4] = Nodo(4)
Nodos[5] = Nodo(5)
Nodos['t'] = Nodo('t')


In [69]:
Nodos['s'].add_branch(Nodos[2], 7.0)
Nodos['s'].add_branch(Nodos[3], 1.0)

Nodos[2].add_branch(Nodos[4], 2.0)
Nodos[2].add_branch(Nodos['t'], 3.0)

Nodos[3].add_branch(Nodos[5], 2.0)

Nodos[4].add_branch(Nodos['t'], 1.0)

Nodos[5].add_branch(Nodos['t'], 5.0)


### Algorithm implementation

In [70]:
def find_available_path(origin, destination, flow = None):
    found = None
    for nodo, parameters in origin.branches.items():
        if parameters['available_flow'] > 0:  #There are remaining flow
            if nodo == destination:
                #found a path
                n_flow = parameters['available_flow']
                found = nodo
                #What is the available flow?
                if not flow or n_flow < flow:    #flow can be None at the start. Also can be less that previous
                    flow = n_flow
                
            else:  #node not destination node, try recursive search on this branch, 
                n_flow = parameters['available_flow']
                if flow and flow < n_flow:
                    n_flow = flow                   #n_flow is the max. available flow in this path up to here.
                
                found, n_flow = find_available_path(nodo, destination, n_flow)
                if found:  #We've found a brach that connects with destination
                    if not flow or n_flow < flow:  
                        flow = n_flow

        if found:
            #We've found destination, now I update node flow stats:
            parameters['available_flow'] = parameters['available_flow'] - flow
            parameters['current_flow'] = parameters['current_flow'] + flow
            #end branch iteration:
            break 
    
    return found, flow

In [71]:
def Ford_Fulkenson(origin, destination):
    while find_available_path(origin, destination)[0]:
        pass

### Resolution

In [72]:
Ford_Fulkenson(Nodos['s'], Nodos['t'])

### Results

In [73]:
print (Nodos['s'].report())

Node s - Branches: 2
    Branch Node 2  - Max Flow 7.0 - Current Flow 4.0 - Available Flow 3.0
    Branch Node 3  - Max Flow 1.0 - Current Flow 1.0 - Available Flow 0.0
Node Stats: Max Flow 8.0 - Current Flow 5.0 - Available Flow 3.0



## Ex 6 - Transport problem

I repeat procedures used on problems 2 and 4.

### Problem data

I will use more descriptive nodes in order to facilitate working with indexes.

In [74]:
named_nodes = ['of1', 'of2', 'of3', 'd1', 'd2']   #node enumeration. Can be any non repeated inmutable.

In [75]:
nodes = nodes_indexer(named_nodes)

On each arc I will put cost of transportation

In [76]:
graph = []

graph.append(((nodes['of1'], nodes['d1']), 10))
graph.append(((nodes['of1'], nodes['d2']), 20))

graph.append(((nodes['of2'], nodes['d1']), 10))
graph.append(((nodes['of2'], nodes['d2']), 10))

graph.append(((nodes['of3'], nodes['d1']), 10))
graph.append(((nodes['of3'], nodes['d2']), 30))

graph

[((0, 3), 10),
 ((0, 4), 20),
 ((1, 3), 10),
 ((1, 4), 10),
 ((2, 3), 10),
 ((2, 4), 30)]

Suply and demands:

In [77]:
b= np.zeros((len(nodes)))
b[nodes['of1']] = 10
b[nodes['of2']] = 20
b[nodes['of3']] = 15

b[nodes['d1']] = -25
b[nodes['d2']] = -20

b

array([ 10.,  20.,  15., -25., -20.])

Limits:

In [78]:
limites = [(0, None) for a in graph]
limites

[(0, None), (0, None), (0, None), (0, None), (0, None), (0, None)]

### Procedure

Arc-Nodes matrix construction:

In [79]:
res = ANMatrix_from_arcs(graph, len(nodes))
A = res[0]
arcos = res[1]

In [80]:
A

array([[ 1,  1,  0,  0,  0,  0],
       [ 0,  0,  1,  1,  0,  0],
       [ 0,  0,  0,  0,  1,  1],
       [-1,  0, -1,  0, -1,  0],
       [ 0, -1,  0, -1,  0, -1]])

Cost function coefficients:

In [81]:
c = [a[1] for a in graph]
c

[10, 20, 10, 10, 10, 30]

I will run optimization algorithm:

In [82]:
r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, method='revised simplex')

<ipython-input-82-62916b0d23f2>:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  r = linprog(c=c, A_eq = A, b_eq = b, bounds = limites, method='revised simplex')


Status:

In [83]:
r.success

True

Total transportation cost:

In [84]:
r.fun

450.0

Quantities transported on each route:

In [85]:
x = r.x

reversed_nodes = dict([(nodes[k], k) for k in nodes.keys()])

In [86]:
for a in range(len(graph)):
    print("De nodo {0} a nodo {1}: {2}".format(reversed_nodes[graph[a][0][0]], reversed_nodes[graph[a][0][1]], x[a]))

De nodo of1 a nodo d1: 10.0
De nodo of1 a nodo d2: 0.0
De nodo of2 a nodo d1: 0.0
De nodo of2 a nodo d2: 20.0
De nodo of3 a nodo d1: 15.0
De nodo of3 a nodo d2: 0.0
